In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from time_series_transform.sequence_transfomer import *
from time_series_transform.time_series_transformer import Time_Series_Transformer

In [4]:
df = pd.read_csv('../Data/Train/sales_train_validation.csv').sample(2)
dimList = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
df = df.drop(dimList+['id'],axis =1)
dimList = []

In [28]:
tst = Time_Series_Transformer(df,dimList)
dataset,step = tst.get_tf_dataset(10)

In [20]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _tensor_feature(value):
    return _bytes_feature(tf.io.serialize_tensor(value))

In [21]:
def serialize_example(X,y):
  """
  Creates a tf.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.
  for i in X:
    feature = {
        i:_tensor_feature(X[i]),
        'label':_float_feature(y)
    }
  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [22]:
def tf_serialize_example(X,y):
  tf_string = tf.py_function(
    serialize_example,
    (X,y),  # pass these args to the above function.
    tf.string)      # the return type is `tf.string`.
  return tf.reshape(tf_string, ()) # The result is a scalar

In [34]:
serialized_features_dataset = dataset.map(tf_serialize_example)
serialized_features_dataset

TypeError: in converted code:

    <ipython-input-22-b67d32025816>:2 tf_serialize_example  *
        tf_string = tf.py_function(
    C:\Users\USER\Anaconda3\lib\site-packages\tensorflow_core\python\ops\script_ops.py:407 eager_py_func
        return _internal_py_func(func=func, inp=inp, Tout=Tout, eager=True, name=name)
    C:\Users\USER\Anaconda3\lib\site-packages\tensorflow_core\python\ops\script_ops.py:296 _internal_py_func
        input=inp, token=token, Tout=Tout, name=name)
    C:\Users\USER\Anaconda3\lib\site-packages\tensorflow_core\python\ops\gen_script_ops.py:74 eager_py_func
        "EagerPyFunc", input=input, token=token, Tout=Tout, name=name)
    C:\Users\USER\Anaconda3\lib\site-packages\tensorflow_core\python\framework\op_def_library.py:502 _apply_op_helper
        "%s that are invalid. Tensors: %s" % (prefix, values))

    TypeError: Tensors in list passed to 'input' of 'EagerPyFunc' Op have types [<NOT CONVERTIBLE TO TENSOR>, float32] that are invalid. Tensors: ({'time_s

In [27]:
dataset.shuffle(10)

AttributeError: 'tuple' object has no attribute 'shuffle'